# DLS project by Alexandr Korchemnyi
# Файлы проекта загружены на [GitHub](https://github.com/Yessense/yolov5n_detect_drone).


In [17]:
# !git clone https://github.com/ultralytics/yolov5

In [2]:
%cd yolov5
# %pip install -qr requirements.txt

/home/yessense/projects/dls project/yolov5


In [1]:
import torch
import os
from IPython.display import Image, clear_output  # to display images

In [2]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.8.1+cu102 (NVIDIA GeForce RTX 2060)


In [3]:
dataset_directory = "/home/yessense/projects/dls project/dataset"

In [4]:
train_len = len(os.listdir(os.path.join(dataset_directory, "train/images")))
test_len = len(os.listdir(os.path.join(dataset_directory, "test/images")))
valid_len = len(os.listdir(os.path.join(dataset_directory, "valid/images")))

print(f'Lenght of train dataset: {train_len}')
print(f'Lenght of validation dataset: {valid_len}')
print(f'Lenght of test dataset: {test_len}')
print(f'Total lenght with reflect augmentation: {sum([train_len, valid_len, test_len])}')


Lenght of train dataset: 1500
Lenght of validation dataset: 428
Lenght of test dataset: 211
Total lenght with reflect augmentation: 2139


# Данные
Набор данных снят на полигоне для дронов в МИРЭА. Размечен с помощью roboflow.
Тестовый набор данных выложен на Github
[https://github.com/Yessense/yolov5n_detect_drone/tree/master/test/images](https://github.com/Yessense/yolov5n_detect_drone/tree/master/test/images)

![](imgs/train_batch0.jpg)

# Тренировка модели
Изначальная модель - yolov5n
Взята, как самая маленькая, чтобы запускать на слабом компьютере дрона.

In [ ]:
!python train.py --img 320 --batch 16 --epochs 150 --data "/home/yessense/projects/dls project/dataset/data.yaml" --weights yolov5n.pt --cache --hyp "/home/yessense/projects/dls project/dataset/hyp.yaml"

![](imgs/F1_curve.png)
![](imgs/P_curve.png)
![](imgs/PR_curve.png)
![](imgs/R_curve.png)

# Экспорт модели в формат onnx для использования в opencv на дроне

In [ ]:
!python export.py --img 320 --batch 1 --weights "/home/yessense/projects/dls project/yolov5/runs/train/exp5/weights/best.pt" --include onnx

# Код для проверки изображений на дроне

In [ ]:
import os

import cv2
import numpy as np

from cv_bridge import CvBridge
import rospy
from sensor_msgs.msg import Image

# Constants.
INPUT_WIDTH = 320
INPUT_HEIGHT = 320
SCORE_THRESHOLD = 0.5
NMS_THRESHOLD = 0.1
CONFIDENCE_THRESHOLD = 0.4

# Text parameters.
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
THICKNESS = 1

# Colors.
BLACK = (0, 0, 0)
BLUE = (255, 178, 50)
YELLOW = (0, 255, 255)

ros_image = None
bridge = CvBridge()

def callback(message):
    global ros_image
    ros_image = cv2.cvtColor(bridge.imgmsg_to_cv2(message, desired_encoding='passthrough'), cv2.COLOR_BGR2RGB)

def draw_label(im, label, x, y):
    """Draw text onto image at location."""
    # Get text size.
    text_size = cv2.getTextSize(label, FONT_FACE, FONT_SCALE, THICKNESS)
    dim, baseline = text_size[0], text_size[1]
    # Use text size to create a BLACK rectangle.
    cv2.rectangle(im, (x, y), (x + dim[0], y + dim[1] + baseline), (0, 0, 0), cv2.FILLED);
    # Display text inside the rectangle.
    cv2.putText(im, label, (x, y + dim[1]), FONT_FACE, FONT_SCALE, YELLOW, THICKNESS, cv2.LINE_AA)


def pre_process(input_image, net):
    # Create a 4D blob from a frame.
    blob = cv2.dnn.blobFromImage(input_image, 1 / 255, (INPUT_WIDTH, INPUT_HEIGHT), [0, 0, 0], 1, crop=False)

    # Sets the input to the network.
    net.setInput(blob)

    # Run the forward pass to get output of the output layers.
    outputs = net.forward(net.getUnconnectedOutLayersNames())
    return outputs


def post_process(input_image, outputs, draw=False):
    # Lists to hold respective values while unwrapping.
    class_ids = []
    confidences = []
    boxes = []
    # Rows.
    rows = outputs[0].shape[1]
    image_height, image_width = input_image.shape[:2]
    # Resizing factor.
    x_factor = image_width / INPUT_WIDTH
    y_factor = image_height / INPUT_HEIGHT
    # Iterate through detections.
    for r in range(rows):
        row = outputs[0][0][r]
        confidence = row[4]
        # Discard bad detections and continue.
        if confidence >= CONFIDENCE_THRESHOLD:
            classes_scores = row[5:]
            # Get the index of max class score.
            class_id = np.argmax(classes_scores)
            #  Continue if the class score is above threshold.
            if (classes_scores[class_id] > SCORE_THRESHOLD):
                confidences.append(confidence)
                class_ids.append(class_id)
                cx, cy, w, h = row[0], row[1], row[2], row[3]
                left = int((cx - w / 2) * x_factor)
                top = int((cy - h / 2) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])
                boxes.append(box)
                # Perform non maximum suppression to eliminate redundant, overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)

    if draw:
        for i in indices:
            box = boxes[i]
            left = box[0]
            top = box[1]
            width = box[2]
            height = box[3]
            # Draw bounding box.
            cv2.rectangle(input_image, (left, top), (left + width, top + height), BLUE, 3 * THICKNESS)
            # Class label.
            label = "{}:{:.2f}".format(classes[class_ids[i]], confidences[i])
            # Draw label.
            draw_label(input_image, label, left, top - 30)
    else:
        input_image = None
    if not len(indices):
        return input_image, [], 0
    else:
        index = np.argmax(confidences)
        return input_image, boxes[index], confidences[index]


if __name__ == '__main__':
    rospy.init_node("drone_detection")
    draw = True

    # capture = cv2.VideoCapture(0)
    bridge = CvBridge()
    # capture.set(3, 640)
    # capture.set(4, 480)
    topic = rospy.Publisher("camera/color/drone_detection", Image, queue_size=10)
    rospy.Subscriber("/camera/color/image_raw", Image, callback)

    modelWeights = "weights.onnx"
    net = cv2.dnn.readNet(modelWeights)
    print("Net initialized")

    # Load class names.
    classesFile = "classes.txt"
    with open(classesFile, 'rt') as f:
        classes = f.read().rstrip('\n').split('\n')

    images_dir = './test/images/'
    # for image in os.listdir(images_dir):
    while True:
        # image_path = os.path.join(images_dir, image)
        # Load image.
        # frame = cv2.imread(image_path)
        # success, frame = capture.read()
        frame = ros_image
        if frame is None:
            print("NULL FRAME")
            continue
        print("New frame")
        # Give the weight files to the model and load the network using       them.
        # Process image.
        detections = pre_process(frame, net)
        img, boxes, confidences = post_process(frame.copy(), detections, draw=draw)
        print(boxes)
        """
        Put efficiency information. The function getPerfProfile returns       the overall time for results(t)
        and the timings for each of the layers(in layersTimes).
        """
        t, _ = net.getPerfProfile()
        label = 'Inference time: %.2f ms' % (t * 1000.0 / cv2.getTickFrequency())

        if img is not None:
            print("IMG IS NOT NONE")
            cv2.putText(img, label, (20, 40), FONT_FACE, FONT_SCALE,  (0, 0, 255), THICKNESS, cv2.LINE_AA)
            # cv2.imshow('Output', img)
            image_message = bridge.cv2_to_imgmsg(img, encoding="bgr8")
            topic.publish(image_message)

            cv2.waitKey(300)

# Результаты
## Загружены файлы на дрон
![](results/1.jpeg)
## Дрон перехватчик с камерой, направленной на дрона-жертву
Камера intel realsense
![](results/2.jpeg)
## Запуск нейросети на дроне
На дроне стоит слабый компьютер raspberry pi 4, поэтому взята самая слабая версия yolo.
![](results/3.jpeg)
## Дрон-жертва стабильно распознается
Промежуток распознавания 1 секунда, потому что еще выводятся изображения в ros топик и на них рисуются bounding box'ы.
![](results/4.jpeg)
## Дрон
https://clover.coex.tech/ru/
![](results/5.jpeg)
## Перехват дрона-жертвы с автоматическим наведением с помощью натренированной нейросети.
![](results/find.gif)

